# [Learn Quantum Computing with Python and Q#](https://www.manning.com/books/learn-quantum-computing-with-python-and-q-sharp?a_aid=learn-qc-granade&a_bid=ee23f338)
# Chapter 12 | Arithmetic with quantum computers
----
> Copyright (c) Sarah Kaiser and Chris Granade.
> Code sample from the book "Learn Quantum Computing with Python and Q#" by
> Sarah Kaiser and Chris Granade, published by Manning Publications Co.
> Book ISBN 9781617296130.
> Code licensed under the MIT License.

# Arithmetic samples

For this section of the notebook, we will turn off displaying states with small amplitudes so that when we are looking at larger registers it is easy to see the data we are looking for.

In [1]:
%config dump.truncateSmallAmplitudes = "true"

"true"

The `Microsoft.Quantum.Arithmetic` namespace is provided by the Q# Numerics package, and we want to tell the Q# kernel here to add that package to our workspace.

In [2]:
%package Microsoft.Quantum.Numerics

Adding package Microsoft.Quantum.Numerics: done!

Microsoft.Quantum.Standard::0.17.2105143879, Microsoft.Quantum.Standard.Visualization::0.17.2105143879, Microsoft.Quantum.Numerics::0.17.2105143879

### Sample 1: Adding integers with qubits

In [3]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation AddCustom(num1 : Int, num2 : Int) : Int {
    // We need the registers to be large enough to hold the max possible sum
    let bitSize = BitSizeI(MaxI((num1, num2))) + 1;
    use (reg1, reg2) = (Qubit[bitSize], Qubit[bitSize]);
        
    // Here we configure two registers with LittleEndian encoding
    let qubits1 = LittleEndian(reg1);
    let qubits2 = LittleEndian(reg2);
        
    // Encode the integers in the qubit registers
    ApplyXorInPlace(num1, qubits1);
    ApplyXorInPlace(num2, qubits2);
        
    Message("Before addition:");
    DumpRegister((),reg2);
        
    // Using the Numerics package operation `AddI` you can add integers 
    // represented by two input registers `qubits1` and `qubits2`
    AddI(qubits1, qubits2);

    Message("After addition:");
    DumpRegister((),reg2);
        
    // Lastly, you reset the first register and measure the register
    // with the results. `MeasureInteger` also resets the register 
    // after it is done.
    ResetAll(reg1);
    return MeasureInteger(qubits2);
}

AddCustom

In [4]:
%simulate AddCustom num1=4 num2=6

Before addition:


|6⟩	1 + 0𝑖

After addition:


|10⟩	1 + 0𝑖

10

### Sample 2: Multiplying 

In [5]:
open Microsoft.Quantum.Arrays;

// This operation takes a register and a pair of integers
// and prepares that register in a superposition of those
// integers in a LittleEndian encoding.
operation PrepareSuperpositionOfTwoInts(register : LittleEndian, intPair : (Int, Int)) 
: Unit is Adj + Ctl {
    use ctrl = Qubit();
    // The hadamard operation puts our control qubit in a superposition,
    // so that when we control later operations on it they are also in superposition.
    H(ctrl);
        
    within{
        X(ctrl);
    } 
    apply{
        // Adding the `Controlled` functor here adds a first argument of 
        // the control register and then the arguments for the regular operation.
        // Here we are taking the first int of the pair and ensuring that the 
        // register is encoded as LittleEndian.
        Controlled ApplyXorInPlace([ctrl], (Fst(intPair), register));
    }
        
    // Now we can do the same thing with the second int in `intPair` and encode
    // it in `register` controlled on the `ctl` qubit.
    Controlled ApplyXorInPlace([ctrl], (Snd(intPair), register));
    // Here we add some unnecessary phase rotation so we can see how
    // that propagates through later steps.
    (ControlledOnInt(Snd(intPair), Y))(register!, ctrl);  
}


operation MultiplyInSuperpositionMod( superpositionInt1 : Int,
    superpositionInt2 : Int, multiplier : Int, modulus : Int) 
: Int {
    // The first thing here is to figure out how big we need our
    // registers to be, and allocate a register of qubits of that size.
    // Here since we are doing modular multiplication, the largest possible 
    // value our register has to hold is the modulus - 1.
    use target = Qubit[BitSizeI(modulus)];
    // We can use the other operation defined in this cell to prepare
    // our target register in a superposition of integers.
    PrepareSuperpositionOfTwoInts(LittleEndian(target), (superpositionInt1, superpositionInt2));
        
    Message("Before multiplication:");
    DumpMachine();
        
    // The Numerics package provides an operation that can take a 
    // LittleEndian register and multiply the value it represents by a
    // multiplier mod the modulus.
    MultiplyByModularInteger(multiplier, modulus, LittleEndian(target));
        
    Message("After multiplication:");
    DumpMachine();
    return MeasureInteger(LittleEndian(target));
}

PrepareSuperpositionOfTwoInts, MultiplyInSuperpositionMod

Here we are multiplying 3 by a superposition of 2 and 3 all mod 8.

In [6]:
%simulate MultiplyInSuperpositionMod superpositionInt1=2 superpositionInt2=3 multiplier=3 modulus=8

Before multiplication:


|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0 + -0.7071067811865476𝑖

After multiplication:


|1⟩	4.895614577113624E-17 + -0.707106781186565𝑖
|6⟩	0.7071067811865637 + -5.144551441211654E-16𝑖

1

----

# Factoring samples

In [7]:
%simulate FactorSemiprimeInteger number=21

We have guessed a divisor of 21 to be 3 by accident. Nothing left to do.


(3, 7)

In [8]:
%simulate FactorSemiprimeInteger number=21

Estimating period of 5...
The estimated period did not yield a valid factor, trying again.
We have guessed a divisor of 21 to be 3 by accident. Nothing left to do.


(3, 7)

----
### Epilogue

_The following cell logs what version of the components this was last tested with._

In [9]:
%version

Component,Version
iqsharp,0.17.2105143879
Jupyter Core,1.5.0.0
.NET Runtime,".NETCoreApp,Version=v3.1"
